In [1]:
import pcmdi_metrics
json1 = 'gfdl_080917_r1i1p1/rt_2.5x2.5_esmf_conservative_metrics.json'
json2 = 'gfdl_080917_r2i1p1/rt_2.5x2.5_esmf_conservative_metrics.json'

In [2]:
J1 = pcmdi_metrics.io.base.JSONs([json1])

In [3]:
J1.getAxisIds()

['variable',
 u'model',
 u'reference',
 u'rip',
 u'region',
 u'statistic',
 u'season']

In [4]:
print J1.getAxis("statistic")

   id: statistic
   Length: 16
   First:  bias_xy
   Last:   std_xyt
   Python id:  0x7fdca05b8b10



In [5]:
print J1.getAxis("statistic")[:]

[u'bias_xy' u'cor_xy' u'mae_xy' u'mean-obs_xy' u'mean_xy' u'rms_devzm'
 u'rms_xy' u'rms_xyt' u'rms_y' u'rmsc_xy' u'std-obs_xy' u'std-obs_xy_devzm'
 u'std-obs_xyt' u'std_xy' u'std_xy_devzm' u'std_xyt']


In [6]:
print J1.getAxis("model")[:]

[u'ACCESS1-0' u'ACCESS1-3' u'BNU-ESM' u'CCSM4' u'CESM1-CAM5' u'CMCC-CM'
 u'CNRM-CM5' u'CSIRO-Mk3-6-0' u'CanAM4' u'FGOALS-s2' u'GFDL-CM3'
 u'GISS-E2-R' u'HadGEM2-A' u'IPSL-CM5A-LR' u'IPSL-CM5A-MR' u'IPSL-CM5B-LR'
 u'MIROC5' u'MPI-ESM-LR' u'MPI-ESM-MR' u'NorESM1-M' u'bcc-csm1-1'
 u'bcc-csm1-1-m' u'inmcm4']


In [7]:
data = J1(statistic=['rms_xy','std_xy'],model=['ACCESS1-0','CNRM-CM5','FGOALS-s2'])
print data.shape

(1, 3, 2, 1, 10, 2, 5)


In [8]:
print data.getAxis(-2)  # Now its' an MV can't get Axis by name

   id: statistic
   Length: 2
   First:  rms_xy
   Last:   std_xy
   Python id:  0x7fdca0591d50



In [9]:
print data.getAxisList()

[   id: variable
   Length: 1
   First:  rt
   Last:   rt
   Python id:  0x7fdca05b8910
,    id: model
   Length: 3
   First:  ACCESS1-0
   Last:   FGOALS-s2
   Python id:  0x7fdca0591790
,    id: reference
   Length: 2
   First:  SimulationDescription
   Last:   defaultReference
   Python id:  0x7fdca05b86d0
,    id: rip
   Length: 1
   First:  r1i1p1
   Last:   r1i1p1
   Python id:  0x7fdca05b82d0
,    id: region
   Length: 10
   First:  20N50N
   Last:   ocean
   Python id:  0x7fdca05b8210
,    id: statistic
   Length: 2
   First:  rms_xy
   Last:   std_xy
   Python id:  0x7fdca0591d50
,    id: season
   Length: 5
   First:  ann
   Last:   son
   Python id:  0x7fdca05b8b50
]


In [10]:
J12 = pcmdi_metrics.io.base.JSONs([json1,json2])

In [11]:
J12.getAxisIds()

['variable',
 u'model',
 u'reference',
 u'rip',
 u'region',
 u'statistic',
 u'season']

In [12]:
J12.getAxis('rip')

   id: rip
   Length: 2
   First:  r1i1p1
   Last:   r2i1p1
   Python id:  0x7fdca05e4a50

In [13]:
data2 = J12(statistic=['rms_xy','std_xy'],model=['ACCESS1-0','CNRM-CM5','FGOALS-s2'])
print data2.shape

(1, 3, 2, 2, 10, 2, 5)
